In [2]:
!cp "/content/drive/My Drive/cv2_gpu/cv2.cpython-37m-x86_64-linux-gnu.so" .

In [3]:
import cv2

In [4]:
cv2.__version__

'4.5.5-dev'

In [5]:
import os
import numpy as np
import time
import pandas as pd
from statistics import mean
from IPython.display import Image

In [17]:
lst_input = [
    "/content/drive/MyDrive/Colab Notebooks/video/240p_60fps.mp4",
    "/content/drive/MyDrive/Colab Notebooks/video/360p_30fps.mp4",
    "/content/drive/MyDrive/Colab Notebooks/video/480p_30fps.mp4",
    "/content/drive/MyDrive/Colab Notebooks/video/720p_30fps.mp4",
    "/content/drive/MyDrive/Colab Notebooks/video/1080p_30fps.mp4",
    "/content/drive/MyDrive/Colab Notebooks/video/1080p_60fps.mp4"
]

In [23]:
def get_fps(network, path_to_onnx, input_list, use_cuda):
    os.environ["DISPLAY"] = ':0'
    dataframe = pd.DataFrame(columns=["Input", "Output", "Video"], index=input_list)

    for video in input_list:
        print("[INFO] loading model...")
        net = cv2.dnn.readNetFromONNX(path_to_onnx)

        #comment this two lines to execute code only with cpu
        if use_cuda:
          net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
          net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

        time_elap = []
        fps_output_list = []

        timeStamp = time.time()
        fpsFilter = 0
        font = cv2.FONT_HERSHEY_SIMPLEX

        frames = 0

        if video[-1] == "0":  # for webcam0 (integrated)
            # width = 1280
            # height = 720
            flip = 2
            dispW = 640
            dispH = 480
            camSet = 'nvarguscamerasrc !  video/x-raw(memory:NVMM), width=1280, height=720, format=NV12, framerate=30/1 ! nvvidconv flip-method=' + str(
                flip) + ' ! video/x-raw, width=' + str(dispW) + ', height=' + str(
                dispH) + ', format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink'
            total_num_frames = 300
            fps_input = 30
            cap = cv2.VideoCapture(camSet)
        elif video[-1] == "1":  # for webcam1 (logitech)
            total_num_frames = 300
            fps_input = 30
            cap = cv2.VideoCapture(1)
        else:  # for video.mp4
            cap = cv2.VideoCapture(video)
            total_num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if str(total_num_frames)[0] == "3":
                fps_input = 30
            else:
                fps_input = 60

        #fps_inference = FPS().start()
        while frames <= total_num_frames:
            (grabbed, frame) = cap.read()
            frames += 1
            print("Current frame: ", frames, " on: ", total_num_frames, " network:", network, " Source: ", video)
            if not grabbed:
                break
            if video[-1] != "4":
                frame = im.resize(frame, width=1280)

            h, w = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
            net.setInput(blob)

            start = time.time()
            # inference
            detections = net.forward()
            #fps_inference.update()  # fps INFERENCE
            end = time.time()

            # compute the fps inference
            elap = (end - start)
            time_elap.append(elap)

            CLASSES = ["background", "bicycle", "bus", "car", "motorcycle", "person", "traffic light", "traffic sign",
                       "truck"]
            COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

            for i in np.arange(detections.shape[2]):
                confidence = detections[0, i, 2]
                if confidence > 0.5:
                    idx = int(detections[0, 0, i, 1])
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")
                    label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
                    cv2.rectangle(frame, (startX, startY), (endX, endY), COLORS[idx], 2)
                    y = startY - 15 if startY - 15 > 15 else startY + 15
                    cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

            dt = time.time() - timeStamp
            timeStamp = time.time()
            fps = 1 / dt
            fpsFilter = .9 * fpsFilter + .1 * fps
            fps_output_list.append(fpsFilter)

            Image(detections)

        try:
            fps_inference = 1 / (sum(time_elap) / len(time_elap))
        except:
            print("Division by zero")

        print("[INFO] FPS INFERENCE: {:.2f}".format(fps_inference))
        fps_output = mean(fps_output_list)
        print("[INFO] FPS OUTPUT: {:.2f}".format(fps_output))
        print("[INFO] cleaning up...")
        cap.release()

        dataframe.iloc[dataframe.index.get_loc(video), dataframe.columns.get_loc("Input")] = fps_input
        dataframe.iloc[dataframe.index.get_loc(video), dataframe.columns.get_loc("Output")] = fps_output
        dataframe.iloc[dataframe.index.get_loc(video), dataframe.columns.get_loc("Video")] = fps_inference

        print(dataframe)
        #dataframe.to_csv(path)
    return dataframe

In [ ]:
lst_data_dist = []
lst_data_org = []
for i in range(0, 100):
  dataframe_dist = get_fps('SSD Mobilenet-V1', 
                           'drive/MyDrive/Colab Notebooks/models/SSD_distill_Freeze_1000.onnx', 
                           lst_input, 
                           False)
  lst_data_dist.append(dataframe_dist)
  dataframe_org = get_fps('SSD Mobilenet-V1', 
                          'drive/MyDrive/Colab Notebooks/models/teacher-ssd-1000.onnx', 
                          lst_input, 
                          False)
  lst_data_org.append(dataframe_org)

Output streaming troncato alle ultime 5000 righe.
Current frame:  48  on:  312  network: SSD Mobilenet-V1  Source:  /content/drive/MyDrive/Colab Notebooks/video/720p_30fps.mp4
Current frame:  49  on:  312  network: SSD Mobilenet-V1  Source:  /content/drive/MyDrive/Colab Notebooks/video/720p_30fps.mp4
Current frame:  50  on:  312  network: SSD Mobilenet-V1  Source:  /content/drive/MyDrive/Colab Notebooks/video/720p_30fps.mp4
Current frame:  51  on:  312  network: SSD Mobilenet-V1  Source:  /content/drive/MyDrive/Colab Notebooks/video/720p_30fps.mp4
Current frame:  52  on:  312  network: SSD Mobilenet-V1  Source:  /content/drive/MyDrive/Colab Notebooks/video/720p_30fps.mp4
Current frame:  53  on:  312  network: SSD Mobilenet-V1  Source:  /content/drive/MyDrive/Colab Notebooks/video/720p_30fps.mp4
Current frame:  54  on:  312  network: SSD Mobilenet-V1  Source:  /content/drive/MyDrive/Colab Notebooks/video/720p_30fps.mp4
Current frame:  55  on:  312  network: SSD Mobilenet-V1  Source:  /c

In [25]:
def dataframe_bench(path, lst):
  dataframe = pd.DataFrame(columns=["Input", "Output", "Video"], index=lst_input)
  for video in lst_input:
    input = 0
    output = 0
    network = 0
    for i in range(0, len(lst)):
      input += lst[i].iloc[lst[i].index.get_loc(video), lst[i].columns.get_loc("Input")]
      output += lst[i].iloc[lst[i].index.get_loc(video), lst[i].columns.get_loc("Output")]
      network += lst[i].iloc[lst[i].index.get_loc(video), lst[i].columns.get_loc("Video")]

    dataframe.iloc[dataframe.index.get_loc(video), dataframe.columns.get_loc("Input")] = input/len(lst)
    dataframe.iloc[dataframe.index.get_loc(video), dataframe.columns.get_loc("Output")] = output/len(lst)
    dataframe.iloc[dataframe.index.get_loc(video), dataframe.columns.get_loc("Video")] = network/len(lst)
    
  print(dataframe)
  dataframe.to_csv(path)

In [26]:
dataframe_bench('./drive/MyDrive/Colab Notebooks/bench_original_NoCuda.csv', lst_data_org)
dataframe_bench('./drive/MyDrive/Colab Notebooks/bench_distill_NoCuda.csv', lst_data_dist)

                                                   Input  ...       Video
/content/drive/MyDrive/Colab Notebooks/video/24...  60.0  ...  280.521491
/content/drive/MyDrive/Colab Notebooks/video/36...  30.0  ...  297.120839
/content/drive/MyDrive/Colab Notebooks/video/48...  30.0  ...  297.165064
/content/drive/MyDrive/Colab Notebooks/video/72...  30.0  ...  295.780625
/content/drive/MyDrive/Colab Notebooks/video/10...  30.0  ...  246.239518
/content/drive/MyDrive/Colab Notebooks/video/10...  60.0  ...  198.029656

[6 rows x 3 columns]
                                                   Input  ...       Video
/content/drive/MyDrive/Colab Notebooks/video/24...  60.0  ...  504.378924
/content/drive/MyDrive/Colab Notebooks/video/36...  30.0  ...  484.975003
/content/drive/MyDrive/Colab Notebooks/video/48...  30.0  ...  456.824883
/content/drive/MyDrive/Colab Notebooks/video/72...  30.0  ...  408.168855
/content/drive/MyDrive/Colab Notebooks/video/10...  30.0  ...  343.639711
/content/drive/M